In [1]:
import numpy as np
import pandas as pd
import utils

In [2]:
df_train = pd.read_csv("../data/mimic-emr-ext-train-S0.csv")
df_train.drop(columns=["starttime", "endtime"], inplace=True)
label = "duration"
print(df_train.shape)
df_train.head()

(10121, 103)


,stay_id,admission_location,insurance,language,ethnicity,marital_status,gender,age,hours_in_hosp_before_intubation,weight,...,sodium_score,albumin_score,bilirubin_score,glucose_score,acidbase_score,gcs_score,duration,log_duration,over72h,alive96h
0,38910812,EMERGENCY ROOM,Other,ENGLISH,UNKNOWN,SINGLE,M,56,17,77.0,...,2.0,0.0,8.0,0.0,6.0,48.0,75.033333,4.317932,1,0
1,38388229,EMERGENCY ROOM,Other,ENGLISH,BLACK/AFRICAN AMERICAN,MARRIED,M,81,45,95.5,...,0.0,0.0,0.0,3.0,3.0,33.0,213.633333,5.364261,1,1
2,31753166,TRANSFER FROM HOSPITAL,Medicare,ENGLISH,WHITE,MARRIED,M,91,73,79.5,...,0.0,NaN,NaN,0.0,2.0,24.0,90.416667,4.504429,1,1
3,30003299,EMERGENCY ROOM,Other,ENGLISH,WHITE,SINGLE,M,26,1,120.0,...,0.0,NaN,NaN,0.0,12.0,15.0,154.183333,5.038142,1,1
4,31166711,EMERGENCY ROOM,Other,ENGLISH,WHITE,SINGLE,M,42,77,97.6,...,2.0,11.0,5.0,0.0,12.0,3.0,420.283333,6.040929,1,1


In [3]:
features=None

X_train, y_train = utils.get_X_and_y(df_train, features=features, label=label)
print(X_train.shape, y_train.shape)

preprocessor = utils.define_preprocessor(X_train.columns)

(10121, 98) (10121,)


In [4]:
numerical_features = preprocessor.transformers[0][2]
categorical_features = preprocessor.transformers[1][2]
categorical_features

['admission_location',
 'insurance',
 'language',
 'ethnicity',
 'marital_status',
 'gender']

In [5]:
imputer = preprocessor.transformers[1][1][0]
X_cat = imputer.fit_transform(X_train[categorical_features])
encoder = preprocessor.transformers[1][1][1]
encoder.fit(X_cat)
encoded_categorical_features = list(encoder.get_feature_names())

In [6]:
X_train = preprocessor.fit_transform(X_train)
X_train = pd.DataFrame(X_train, columns = numerical_features + encoded_categorical_features)

X_train = pd.concat([X_train, df_train.iloc[:,-4:]], axis=1)

In [8]:
X_train.to_csv("../data/mimic-emr-ext-train-S0-preproc.csv", index=False)

In [9]:
df_test = pd.read_csv("../data/mimic-emr-ext-test-S0.csv")
df_test.drop(columns=["starttime", "endtime"], inplace=True)

print(df_test.shape)
df_test.head()

(2531, 103)


,stay_id,admission_location,insurance,language,ethnicity,marital_status,gender,age,hours_in_hosp_before_intubation,weight,...,sodium_score,albumin_score,bilirubin_score,glucose_score,acidbase_score,gcs_score,duration,log_duration,over72h,alive96h
0,39361378,EMERGENCY ROOM,Other,ENGLISH,UNKNOWN,SINGLE,F,35,2,80.0,...,0.0,6.0,6.0,3.0,5.0,48.0,128.750000,4.857873,1,1
1,30754300,TRANSFER FROM HOSPITAL,Medicare,ENGLISH,WHITE,MARRIED,M,74,0,127.0,...,0.0,NaN,0.0,0.0,12.0,24.0,455.433333,6.121249,1,1
2,31375344,TRANSFER FROM HOSPITAL,Medicare,ENGLISH,UNKNOWN,DIVORCED,M,69,16,70.0,...,0.0,0.0,0.0,8.0,2.0,0.0,24.466667,3.197312,0,1
3,31604434,EMERGENCY ROOM,Medicaid,ENGLISH,OTHER,SINGLE,M,38,2,130.0,...,0.0,NaN,0.0,3.0,3.0,15.0,201.216667,5.304382,1,1
4,38669252,EMERGENCY ROOM,Medicare,ENGLISH,BLACK/AFRICAN AMERICAN,MARRIED,F,42,52,58.7,...,2.0,0.0,0.0,9.0,NaN,15.0,30.366667,3.413346,0,1


In [10]:
X_test, y_test = utils.get_X_and_y(df_test, features=features, label=label)
print(X_test.shape, y_test.shape)

(2531, 98) (2531,)


In [11]:
X_test = preprocessor.transform(X_test)
X_test = pd.DataFrame(X_test, columns = numerical_features + encoded_categorical_features)

X_test = pd.concat([X_test, df_test.iloc[:,-4:]], axis=1)

In [12]:
X_test.to_csv("../data/mimic-emr-ext-test-S0-preproc.csv", index=False)